## Data Shippers

#### Beats is the platform for single-purpose data shippers. They send data from hundreds or thousands of machines and systems to Logstash or Elasticsearch.

#### In this module we will guide the user through the installation of the beats listed below which allos the  collection of various types of host data:

* Filebeat
* Packetbeat
* Metricbeat
* Auditbeat
* Heartbeat

### Filebeat installation

In [ ]:
%%bash
sudo apt install -y filebeat

### Disable default configuration

In [ ]:
%%bash
sudo sed -i '28 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '117 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '148 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '150 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '176,178 s/^/#/' /etc/filebeat/filebeat.yml

### Enable Configuration to Communicate Filebeat with Logstash

##### As we want to establish a connection between filebeat and logstash, we must recall that we have secured all communication with logstash using ssl. Therefore, any beat that seeks to communicate with logstash must provide the certificate we have generated in the logstash node.

##### In this node, the certificate is located in "/etc/pki/tls/certs/logstash-forwarder.crt". Before setting up the necessary configuration for filebeat to connect with logstash, we will first validate if we can stablish a connection to logstash using the following command:

~~~
telnet <LOGSTASH-IP-ADDRESS> 5044
~~~

##### You will get a message stating "Connected to ....."  To exit press CTRL+] and then quit or CTRL+D

##### Now let us validate teh certificate by using the following command:

~~~
curl -v --cacert /etc/pki/tls/certs/logstash-forwarder.crt <LOGSTASH-IP-ADDRESS>:5044
~~~

##### Let us now proceed to set the log files we would like filebeat to track and ship to logstash, set the ip address and port in which logstash will receive our data, and define the location of our certificate that will be used to secure communication between filebeat and logstash.

In [5]:
%%bash

sudo sed -i "28i \ \ \ \ - /var/log/auth.log" /etc/filebeat/filebeat.yml
sudo sed -i "29i \ \ \ \ - /var/log/syslog" /etc/filebeat/filebeat.yml

sudo sed -i '160 s/^#//g' /etc/filebeat/filebeat.yml 

export es_ip="<LOGSTASH-IP-ADDRESS>"

sudo sed -i "162i \ \ hosts: [\"$es_ip:5044\"]" /etc/filebeat/filebeat.yml

sudo sed -i '167i \ \ ssl.certificate_authorities: ["/etc/pki/tls/certs/logstash-forwarder.crt"]' /etc/filebeat/filebeat.yml

### Enabling the Filebeat's system module.

##### Filebeat supports different modules that allows us to collect and parse logs created by Ubuntu's system loggin service. To enable the system module, run the command below:

In [1]:
%%bash
sudo filebeat modules enable system

Module system is already enabled


##### You can verify if the system module has been enables by using the following command:

~~~
sudo filebeat modules list
~~~

### Loading Index templates

##### An index is a collection of documents that share similar characteristics. We will load an index template for documents stored in filebeat indexes that will be applied automatically when a new index is created. Let us list the current templates in elasticsearch by using the following command:

~~~
curl -u elastic:elasticsiem -XGET 'localhost:9200/_template?filter_path=*.version&pretty'
~~~

In [2]:
%%bash
curl -u elastic:elasticsiem -XGET 'localhost:9200/_template?filter_path=*.version&pretty'

{
  ".monitoring-kibana" : {
    "version" : 7000199
  },
  ".management-beats" : {
    "version" : 70000
  },
  ".monitoring-beats" : {
    "version" : 7000199
  },
  ".ml-anomalies-" : {
    "version" : 7020099
  },
  ".kibana_task_manager" : {
    "version" : 7020099
  },
  ".monitoring-alerts-7" : {
    "version" : 7000199
  },
  ".ml-notifications" : {
    "version" : 7020099
  },
  ".ml-state" : {
    "version" : 7020099
  },
  ".ml-meta" : {
    "version" : 7020099
  },
  ".data-frame-notifications-1" : {
    "version" : 7020099
  },
  ".ml-config" : {
    "version" : 7020099
  },
  ".data-frame-internal-1" : {
    "version" : 7020099
  },
  ".monitoring-es" : {
    "version" : 7000199
  },
  ".monitoring-logstash" : {
    "version" : 7000199
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   766  100   766    0     0  69636      0 --:--:-- --:--:-- --:--:-- 69636


##### As we can observe, there are no templates defined for filebeat indexes. Let us load the filebeat template into elasticsearch by using the following command:

In [11]:
%%bash
cd ~
sudo /usr/share/filebeat/bin/filebeat export template > filebeat.template.json -c /etc/filebeat/filebeat.yml
curl -u elastic:elasticsiem -H 'Content-Type: application/json' -XPUT 'localhost:9200/_template/filebeat-7.3.0' -d@/home/ubuntu/filebeat.template.json
curl -u elastic:elasticsiem -XGET 'localhost:9200/_template/' | jq 'keys'
    

{"acknowledged":true}[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-9",
  ".watches",
  "filebeat-7.3.0"
]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  141k  100    21  100  141k    259  1747k --:--:-- --:--:-- --:--:-- 1748k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  106k  100  106k    0     0  8158k      0 --:--:-- --:--:-- --:--:-- 8158k


### Installing Kibana dashboards for Filebeat

In [ ]:
%%bash
sudo filebeat setup -e -E output.logstash.enabled=false -E output.elasticsearch.hosts=['localhost:9200'] -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 -E output.elasticsearch.username=elastic -E output.elasticsearch.password=elasticsiem

### Start Filebeat

In [13]:
%%bash
sudo systemctl start filebeat

### Alternatively, you can run Filebeat in debug mode by using the following command:

~~~
sudo /usr/share/filebeat/bin/filebeat -e -d "publish" -c /etc/filebeat/filebeat.yml
~~~

### Backloging in filebeat


##### In teh case that you change some of the filters that you set on logstash and would like to pre-process your data again, you will need to delete all indexes where your data was stored, stop filebeat, delete filebeat's registry file, and start filebeat again.

##### To delete filebeat registry, you can use the following command:

~~~
sudo rm -R /var/lib/filebeat/registry
~~~

##### Note: Whenever you process again your data, your receiving logs will have a timestamp stating the time the log was received by elasticsearch and not the time at which the log was generated. The timestamp can be replaced by adding a rule on logstash that takes the timestamp stated at the beggining of each log and replace teh default timestamp (receiving time) set by elasticsearch.


### Packetbeat Installation